In [8]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
years = list(range(2023, 2013, -1))
all_matches = []

In [3]:
for year in years:
    print(f"{year-1}-{year}")

2022-2023
2021-2022
2020-2021
2019-2020
2018-2019
2017-2018
2016-2017
2015-2016
2014-2015
2013-2014


In [4]:
standings_url = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

In [ ]:
soup

In [5]:
standings_table = soup.select('table.stats_table')[0]
links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls[3:]

In [5]:
team_tmp = team_urls[5:]
team_tmp

['https://fbref.com/en/squads/d07537b9/2022-2023/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8602292d/2022-2023/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/2022-2023/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/2022-2023/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/2022-2023/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/2022-2023/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2022-2023/Chelsea-Stats',
 'https://fbref.com/en/squads/8cec06e1/2022-2023/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/7c21e445/2022-2023/West-Ham-United-Stats',
 'https://fbref.com/en/squads/4ba7cbea/2022-2023/Bournemouth-Stats',
 'https://fbref.com/en/squads/e4a775cb/2022-2023/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/d3fd31cc/2022-2023/Everton-Stats',
 'https://fbref.com/en/squads/a2d435b3/2022-2023/Leicester-City-Stats',
 'https://fbref.com/en/squads/5bfb9659/2022-2023/Leeds-United-Stats',
 

In [6]:
for team_url in team_tmp:
    all_matches.clear()
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    for year in years:
        # Scores & Fixtures
        try:
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        except ValueError:
            print(f"Could not find Scores & Fixtures for {team_name} in {year-1}-{year}")
            continue
        # Shooting
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "SoT", "PKatt", "Opponent"]], on=["Date", "Opponent"], how='inner')
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = f"{year-1}-{year}"
        team_data["Team"] = team_name
        team_data["url"] = f"https://fbref.com{links[0]}"
        all_matches.append(team_data)

        previous_season = soup.find('div', class_='prevnext').find('a')['href']
        # previous_season = soup.select("a.prev")[0].get("href")
        standings_url = f"https://fbref.com{previous_season}"
        data = requests.get(standings_url)
        time.sleep(2)
    match_df = pd.concat(all_matches)
    match_df.columns = [c.lower() for c in match_df.columns]
    match_df.to_csv(f"{team_name.lower()}.csv", index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'r

Could not find Scores & Fixtures for Brighton and Hove Albion in 2013-2014


C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'r

Could not find Scores & Fixtures for Brentford in 2013-2014


C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'r

...

C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'r

Could not find Scores & Fixtures for Leicester City in 2013-2014


C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'r

Could not find Scores & Fixtures for Leeds United in 2013-2014


C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_8432\595645788.py:18: FutureWarning: Passing literal html to 'r

# Rename folders and file
- e.g. manchester united -> manchester_united

In [7]:
team_urls
team_names = [team_url.split("/")[-1].replace("-Stats", "").replace("-", "_").lower() for team_url in team_urls]
team_names

['manchester_city',
 'arsenal',
 'manchester_united',
 'newcastle_united',
 'liverpool',
 'brighton_and_hove_albion',
 'aston_villa',
 'tottenham_hotspur',
 'brentford',
 'fulham',
 'crystal_palace',
 'chelsea',
 'wolverhampton_wanderers',
 'west_ham_united',
 'bournemouth',
 'nottingham_forest',
 'everton',
 'leicester_city',
 'leeds_united',
 'southampton']

In [9]:
import os

def create_folders(folder_names, parent_folder="raw_data"):
    # Create the parent folder if it doesn't exist
    if not os.path.exists(parent_folder):
        os.makedirs(parent_folder)

    # Iterate over each folder name and create folders within the parent folder
    for name in folder_names:
        folder_path = os.path.join(parent_folder, name)
        # Check if the folder doesn't already exist
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Folder '{name}' created in '{parent_folder}'.")
        else:
            print(f"Folder '{name}' already exists in '{parent_folder}'.")

create_folders(team_names)

Folder 'manchester_city' created in 'raw_data'.
Folder 'arsenal' created in 'raw_data'.
Folder 'manchester_united' created in 'raw_data'.
Folder 'newcastle_united' created in 'raw_data'.
Folder 'liverpool' created in 'raw_data'.
Folder 'brighton_and_hove_albion' created in 'raw_data'.
Folder 'aston_villa' created in 'raw_data'.
Folder 'tottenham_hotspur' created in 'raw_data'.
Folder 'brentford' created in 'raw_data'.
Folder 'fulham' created in 'raw_data'.
Folder 'crystal_palace' created in 'raw_data'.
Folder 'chelsea' created in 'raw_data'.
Folder 'wolverhampton_wanderers' created in 'raw_data'.
Folder 'west_ham_united' created in 'raw_data'.
Folder 'bournemouth' created in 'raw_data'.
Folder 'nottingham_forest' created in 'raw_data'.
Folder 'everton' created in 'raw_data'.
Folder 'leicester_city' created in 'raw_data'.
Folder 'leeds_united' created in 'raw_data'.
Folder 'southampton' created in 'raw_data'.


In [11]:
def rename_files(root_dir):
    for root, dirs, files in os.walk(root_dir):
        for filename in files:
            old_filepath = os.path.join(root, filename)
            new_filename = filename.replace(' ', '_')
            new_filepath = os.path.join(root, new_filename)

            # Rename the file
            os.rename(old_filepath, new_filepath)
            print(f"Renamed '{filename}' to '{new_filename}'.")

# Provide the root directory
root_dir = 'raw_data'

rename_files(root_dir)


Renamed 'arsenal.csv' to 'arsenal.csv'.
Renamed 'aston villa.csv' to 'aston_villa.csv'.
Renamed 'bournemouth.csv' to 'bournemouth.csv'.
Renamed 'brentford.csv' to 'brentford.csv'.
Renamed 'brighton and hove albion.csv' to 'brighton_and_hove_albion.csv'.
Renamed 'chelsea.csv' to 'chelsea.csv'.
Renamed 'crystal palace.csv' to 'crystal_palace.csv'.
Renamed 'everton.csv' to 'everton.csv'.
Renamed 'fulham.csv' to 'fulham.csv'.
Renamed 'leeds united.csv' to 'leeds_united.csv'.
Renamed 'leicester city.csv' to 'leicester_city.csv'.
Renamed 'liverpool.csv' to 'liverpool.csv'.
Renamed 'manchester city.csv' to 'manchester_city.csv'.
Renamed 'manchester united.csv' to 'manchester_united.csv'.
Renamed 'newcastle united.csv' to 'newcastle_united.csv'.
Renamed 'nottingham forest.csv' to 'nottingham_forest.csv'.
Renamed 'southampton.csv' to 'southampton.csv'.
Renamed 'tottenham hotspur.csv' to 'tottenham_hotspur.csv'.
Renamed 'west ham united.csv' to 'west_ham_united.csv'.
Renamed 'wolverhampton wande

In [10]:
import os
def merge_all_file_csv(folder_path, output_file):
    folders = os.listdir(folder_path)
    all_file_paths = [os.path.join(folder_path, folder, f"{folder}.csv") for folder in folders]
    
    # Đọc từng file và lưu vào 1 file csv duy nhất
    all_data = pd.concat([pd.read_csv(file) for file in all_file_paths])
    all_data.to_csv(os.path.join(folder_path, output_file), index=False)

merge_all_file_csv("../raw_data/", output_file="all_clubs.csv")